In [1]:
#import neccesarry libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
%matplotlib inline

In [2]:
# Reading the train dataset
trn = pd.read_csv('train.csv')
  
#Reading the test dataset                  
tst = pd.read_csv('test.csv')                  

In [3]:
trn.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [4]:
tst.head()

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,az,v,n,f,d,t,a,w,0,...,0,0,0,1,0,0,0,0,0,0
1,2,t,b,ai,a,d,b,g,y,0,...,0,0,1,0,0,0,0,0,0,0
2,3,az,v,as,f,d,a,j,j,0,...,0,0,0,1,0,0,0,0,0,0
3,4,az,l,n,f,d,z,l,n,0,...,0,0,0,1,0,0,0,0,0,0
4,5,w,s,as,c,d,y,i,m,0,...,1,0,0,0,0,0,0,0,0,0


In [5]:
   #drop the ID column as index as it is not needed for prediction
trn.drop('ID',inplace=True,axis=1)
tst.drop('ID',inplace=True,axis=1)
         

In [6]:
# Insights:
#----------
# There are 378 cols in train data set, X columns are named as X1, X2 etc.. and target is the Y column
# There are 377 cols in test data set , all columns except the Y column.


# Since there are lot of columns , we can use the below aggregate function to know about the datatypes of columns

dtype_df = trn.dtypes.reset_index()
dtype_df.columns = ["feature name","dtypes"]
dtype_df.groupby("dtypes").agg("count").reset_index()

,dtypes,feature name
0,int64,368
1,float64,1
2,object,8


In [7]:
dtype_df = tst.dtypes.reset_index()
dtype_df.columns = ["feature name","dtypes"]
dtype_df.groupby("dtypes").agg("count").reset_index()

,dtypes,feature name
0,int64,368
1,object,8


In [8]:
# Print the column if it has any null values
for i in trn.columns:
    if trn[i].isnull().sum() > 0 :
        print(trn[i].isnull().sum())

In [9]:
# No features has null values in train data

In [10]:
# Print the column if it has any null values
for i in tst.columns:
    if tst[i].isnull().sum() > 0 :
        print(tst[i].isnull().sum())

In [11]:
# No features has null values in test data

In [12]:
# Checking the unique values for each column in train data
for i in trn.columns:
    print(i,'**',trn[i].unique())

y ** [130.81  88.53  76.26 ...  85.71 108.77  87.48]
X0 ** ['k' 'az' 't' 'al' 'o' 'w' 'j' 'h' 's' 'n' 'ay' 'f' 'x' 'y' 'aj' 'ak' 'am'
 'z' 'q' 'at' 'ap' 'v' 'af' 'a' 'e' 'ai' 'd' 'aq' 'c' 'aa' 'ba' 'as' 'i'
 'r' 'b' 'ax' 'bc' 'u' 'ad' 'au' 'm' 'l' 'aw' 'ao' 'ac' 'g' 'ab']
X1 ** ['v' 't' 'w' 'b' 'r' 'l' 's' 'aa' 'c' 'a' 'e' 'h' 'z' 'j' 'o' 'u' 'p' 'n'
 'i' 'y' 'd' 'f' 'm' 'k' 'g' 'q' 'ab']
X2 ** ['at' 'av' 'n' 'e' 'as' 'aq' 'r' 'ai' 'ak' 'm' 'a' 'k' 'ae' 's' 'f' 'd'
 'ag' 'ay' 'ac' 'ap' 'g' 'i' 'aw' 'y' 'b' 'ao' 'al' 'h' 'x' 'au' 't' 'an'
 'z' 'ah' 'p' 'am' 'j' 'q' 'af' 'l' 'aa' 'c' 'o' 'ar']
X3 ** ['a' 'e' 'c' 'f' 'd' 'b' 'g']
X4 ** ['d' 'b' 'c' 'a']
X5 ** ['u' 'y' 'x' 'h' 'g' 'f' 'j' 'i' 'd' 'c' 'af' 'ag' 'ab' 'ac' 'ad' 'ae'
 'ah' 'l' 'k' 'n' 'm' 'p' 'q' 's' 'r' 'v' 'w' 'o' 'aa']
X6 ** ['j' 'l' 'd' 'h' 'i' 'a' 'g' 'c' 'k' 'e' 'f' 'b']
X8 ** ['o' 'x' 'e' 'n' 's' 'a' 'h' 'p' 'm' 'k' 'd' 'i' 'v' 'j' 'b' 'q' 'w' 'g'
 'y' 'l' 'f' 'u' 'r' 't' 'c']
X10 ** [0 1]
X11 ** [0]
X12 ** [0 1]
X13 **

In [13]:
# Checking the unique values for each column in test data
for i in tst.columns:
    print(i,'**',tst[i].unique())

X0 ** ['az' 't' 'w' 'y' 'x' 'f' 'ap' 'o' 'ay' 'al' 'h' 'z' 'aj' 'd' 'v' 'ak'
 'ba' 'n' 'j' 's' 'af' 'ax' 'at' 'aq' 'av' 'm' 'k' 'a' 'e' 'ai' 'i' 'ag'
 'b' 'am' 'aw' 'as' 'r' 'ao' 'u' 'l' 'c' 'ad' 'au' 'bc' 'g' 'an' 'ae' 'p'
 'bb']
X1 ** ['v' 'b' 'l' 's' 'aa' 'r' 'a' 'i' 'p' 'c' 'o' 'm' 'z' 'e' 'h' 'w' 'g' 'k'
 'y' 't' 'u' 'd' 'j' 'q' 'n' 'f' 'ab']
X2 ** ['n' 'ai' 'as' 'ae' 's' 'b' 'e' 'ak' 'm' 'a' 'aq' 'ag' 'r' 'k' 'aj' 'ay'
 'ao' 'an' 'ac' 'af' 'ax' 'h' 'i' 'f' 'ap' 'p' 'au' 't' 'z' 'y' 'aw' 'd'
 'at' 'g' 'am' 'j' 'x' 'ab' 'w' 'q' 'ah' 'ad' 'al' 'av' 'u']
X3 ** ['f' 'a' 'c' 'e' 'd' 'g' 'b']
X4 ** ['d' 'b' 'a' 'c']
X5 ** ['t' 'b' 'a' 'z' 'y' 'x' 'h' 'g' 'f' 'j' 'i' 'd' 'c' 'af' 'ag' 'ab' 'ac'
 'ad' 'ae' 'ah' 'l' 'k' 'n' 'm' 'p' 'q' 's' 'r' 'v' 'w' 'o' 'aa']
X6 ** ['a' 'g' 'j' 'l' 'i' 'd' 'f' 'h' 'c' 'k' 'e' 'b']
X8 ** ['w' 'y' 'j' 'n' 'm' 's' 'a' 'v' 'r' 'o' 't' 'h' 'c' 'k' 'p' 'u' 'd' 'g'
 'b' 'q' 'e' 'l' 'f' 'i' 'x']
X10 ** [0 1]
X11 ** [0 1]
X12 ** [0 1]
X13 ** [0 1]
X14 ** [0 1]
X1

In [14]:
# Insights:
#----------
# From the above, we can see most of the featues are binary from X10 onwards 
# Many are constants like X11,X93 etc has only values as zeroes, we will confirm this by finding the variance below
# After confirming the variance as zero, We can drop such cols as they will not contribute to the model
# We can also see fetures with zero variance in train are not similar with test data - so here we have to remove the same features from test data as well

In [15]:
# To identify features with 0 variance , Since we could not find variance of categorical features, we do this after label encoding

In [16]:
# Encoding Categorical features in train data - X0
# print(trn['X0'].unique())
from sklearn.preprocessing import LabelEncoder
# enc = LabelEncoder()
# trn['X0'] = enc.fit_transform(trn['X0'])

In [17]:
# Applying same label encoder to test data X0 column
# tst['X0'] = enc.transform(tst['X0'])

In [18]:
# With the above error, we can understand that some of values in X0 column in test set are not presennt in train set 
# and they were not encoded, hence we get error while applying on test

# Now we will identify the uncommon values
set(trn['X0'].unique()) - (set(tst['X0'].unique()))

{'aa', 'ab', 'ac', 'q'}

In [19]:
set(tst['X0'].unique()) - (set(trn['X0'].unique()))

{'ae', 'ag', 'an', 'av', 'bb', 'p'}

In [20]:
# So we get the uncommon values in both train & test data sets. Now we will apply label encoder using the unique values
enc = LabelEncoder()
enc.fit_transform(['k','az','t','al','o','w', 'j', 'h', 's', 'n', 'ay', 'f', 'x', 'y', 'aj', 'ak', 'am',
 'z', 'q', 'at', 'ap', 'v', 'af', 'a', 'e', 'ai', 'd', 'aq', 'c', 'aa', 'ba', 'as', 'i',
 'r', 'b', 'ax', 'bc', 'u', 'ad', 'au', 'm', 'l', 'aw', 'ao', 'ac', 'g', 'ab', 'ae', 'ag', 'an', 'av', 'bb', 'p'])

array([37, 24, 46, 11, 41, 49, 36, 34, 45, 40, 23, 32, 50, 51,  9, 10, 12,
       52, 43, 18, 15, 48,  6,  0, 31,  8, 30, 16, 29,  1, 26, 17, 35, 44,
       25, 22, 28, 47,  4, 19, 39, 38, 21, 14,  3, 33,  2,  5,  7, 13, 20,
       27, 42], dtype=int64)

In [21]:
trn.head()

,y,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,130.81,k,v,at,a,d,u,j,o,0,...,0,0,1,0,0,0,0,0,0,0
1,88.53,k,t,av,e,d,y,l,o,0,...,1,0,0,0,0,0,0,0,0,0
2,76.26,az,w,n,c,d,x,j,x,0,...,0,0,0,0,0,0,1,0,0,0
3,80.62,az,t,n,f,d,x,l,e,0,...,0,0,0,0,0,0,0,0,0,0
4,78.02,az,v,n,f,d,h,d,n,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
# Now tranform the train data using the encoder
trn['X0'] = enc.transform(trn['X0'])

In [23]:
trn.head()

,y,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,130.81,37,v,at,a,d,u,j,o,0,...,0,0,1,0,0,0,0,0,0,0
1,88.53,37,t,av,e,d,y,l,o,0,...,1,0,0,0,0,0,0,0,0,0
2,76.26,24,w,n,c,d,x,j,x,0,...,0,0,0,0,0,0,1,0,0,0
3,80.62,24,t,n,f,d,x,l,e,0,...,0,0,0,0,0,0,0,0,0,0
4,78.02,24,v,n,f,d,h,d,n,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
tst.head()

,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,az,v,n,f,d,t,a,w,0,0,...,0,0,0,1,0,0,0,0,0,0
1,t,b,ai,a,d,b,g,y,0,0,...,0,0,1,0,0,0,0,0,0,0
2,az,v,as,f,d,a,j,j,0,0,...,0,0,0,1,0,0,0,0,0,0
3,az,l,n,f,d,z,l,n,0,0,...,0,0,0,1,0,0,0,0,0,0
4,w,s,as,c,d,y,i,m,0,0,...,1,0,0,0,0,0,0,0,0,0


In [25]:
# Now tranform the test data using the encoder
tst['X0'] = enc.transform(tst['X0'])

In [26]:
tst.head()

,X0,X1,X2,X3,X4,X5,X6,X8,X10,X11,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,24,v,n,f,d,t,a,w,0,0,...,0,0,0,1,0,0,0,0,0,0
1,46,b,ai,a,d,b,g,y,0,0,...,0,0,1,0,0,0,0,0,0,0
2,24,v,as,f,d,a,j,j,0,0,...,0,0,0,1,0,0,0,0,0,0
3,24,l,n,f,d,z,l,n,0,0,...,0,0,0,1,0,0,0,0,0,0
4,49,s,as,c,d,y,i,m,0,0,...,1,0,0,0,0,0,0,0,0,0


In [27]:
## From the above we can see the value 'az' in both train & test data has been decoded as 24. Now this holds good.
# This can be repeated for the columns X1-X6,X8 as well

In [28]:
# Now we will identify the uncommon values for X1 column
print(set(trn['X1'].unique()) - (set(tst['X1'].unique())))
print(set(tst['X1'].unique()) - (set(trn['X1'].unique())))

set()
set()


In [29]:
# From the above we can see there are no uncommon values between train & test data for column X1. So we can directly apply using column
enc = LabelEncoder()
trn['X1'] = enc.fit_transform(trn['X1'])
tst['X1'] = enc.transform(tst['X1'])

In [30]:
# Now we will identify the uncommon values for X1 column
print(set(trn['X2'].unique()) - (set(tst['X2'].unique())))
print(set(tst['X2'].unique()) - (set(trn['X2'].unique())))

{'aa', 'ar', 'c', 'l', 'o'}
{'w', 'u', 'ab', 'aj', 'ad', 'ax'}


In [31]:
# So we have some uncommon values similar to X0 column
print(trn['X2'].unique())
print(tst['X2'].unique())

['at' 'av' 'n' 'e' 'as' 'aq' 'r' 'ai' 'ak' 'm' 'a' 'k' 'ae' 's' 'f' 'd'
 'ag' 'ay' 'ac' 'ap' 'g' 'i' 'aw' 'y' 'b' 'ao' 'al' 'h' 'x' 'au' 't' 'an'
 'z' 'ah' 'p' 'am' 'j' 'q' 'af' 'l' 'aa' 'c' 'o' 'ar']
['n' 'ai' 'as' 'ae' 's' 'b' 'e' 'ak' 'm' 'a' 'aq' 'ag' 'r' 'k' 'aj' 'ay'
 'ao' 'an' 'ac' 'af' 'ax' 'h' 'i' 'f' 'ap' 'p' 'au' 't' 'z' 'y' 'aw' 'd'
 'at' 'g' 'am' 'j' 'x' 'ab' 'w' 'q' 'ah' 'ad' 'al' 'av' 'u']


In [32]:
# Now we define the encoder and fit the values
encX2 = LabelEncoder()
encX2.fit_transform(['at', 'av', 'n', 'e', 'as', 'aq', 'r', 'ai', 'ak', 'm', 'a', 'k','ae', 's', 'f', 'd', 'ag', 
                     'ay', 'ac', 'ap', 'g', 'i', 'aw', 'y', 'b', 'ao', 'al', 'h', 'x', 'au', 't', 'an', 'z', 'ah', 
                     'p', 'am', 'j', 'q', 'af', 'l', 'aa', 'c', 'o', 'ar','ad', 'ax', 'u', 'ab', 'w', 'aj'])

array([20, 22, 38, 29, 19, 17, 42,  9, 11, 37,  0, 35,  5, 43, 30, 28,  7,
       25,  3, 16, 31, 33, 23, 48, 26, 15, 12, 32, 47, 21, 44, 14, 49,  8,
       40, 13, 34, 41,  6, 36,  1, 27, 39, 18,  4, 24, 45,  2, 46, 10],
      dtype=int64)

In [33]:
# Now tranform the train & test data using the encoder
trn['X2'] = encX2.transform(trn['X2'])
tst['X2'] = encX2.transform(tst['X2'])

In [34]:
# Now we will identify the uncommon values for X3 column
print(set(trn['X3'].unique()) - (set(tst['X3'].unique())))
print(set(tst['X3'].unique()) - (set(trn['X3'].unique())))

set()
set()


In [35]:
# From the above we can see there are no uncommon values between train & test data for column X3. So we can directly apply using column
encX3 = LabelEncoder()
trn['X3'] = encX3.fit_transform(trn['X3'])
tst['X3'] = encX3.transform(tst['X3'])

In [36]:
# Now we will identify the uncommon values for X4 column
print(set(trn['X4'].unique()) - (set(tst['X4'].unique())))
print(set(tst['X4'].unique()) - (set(trn['X4'].unique())))

set()
set()


In [37]:
# From the above we can see there are no uncommon values between train & test data for column X4. So we can directly apply using column
encX4 = LabelEncoder()
trn['X4'] = encX4.fit_transform(trn['X4'])
tst['X4'] = encX4.transform(tst['X4'])

In [38]:
# Now we will identify the uncommon values for X5 column
print(set(trn['X5'].unique()) - (set(tst['X5'].unique())))
print(set(tst['X5'].unique()) - (set(trn['X5'].unique())))

{'u'}
{'b', 'a', 't', 'z'}


In [39]:
# So we have some uncommon values similar to X5 column
trn['X5'].unique()

array(['u', 'y', 'x', 'h', 'g', 'f', 'j', 'i', 'd', 'c', 'af', 'ag', 'ab',
       'ac', 'ad', 'ae', 'ah', 'l', 'k', 'n', 'm', 'p', 'q', 's', 'r',
       'v', 'w', 'o', 'aa'], dtype=object)

In [40]:
# Now we define the encoder and fit the values of column X5
encX5 = LabelEncoder()
encX5.fit_transform(['u', 'y', 'x', 'h', 'g', 'f', 'j', 'i', 'd', 'c', 'af', 'ag', 'ab',
       'ac', 'ad', 'ae', 'ah', 'l', 'k', 'n', 'm', 'p', 'q', 's', 'r',
       'v', 'w', 'o', 'aa','z', 'a', 'b', 't'])

array([27, 31, 30, 14, 13, 12, 16, 15, 11, 10,  6,  7,  2,  3,  4,  5,  8,
       18, 17, 20, 19, 22, 23, 25, 24, 28, 29, 21,  1, 32,  0,  9, 26],
      dtype=int64)

In [41]:
# Now tranform the train & test data using the encoder
trn['X5'] = encX5.transform(trn['X5'])
tst['X5'] = encX5.transform(tst['X5'])

In [42]:
# Now we will identify the uncommon values for X6 column
print(set(trn['X6'].unique()) - (set(tst['X6'].unique())))
print(set(tst['X6'].unique()) - (set(trn['X6'].unique())))

set()
set()


In [43]:
# From the above we can see there are no uncommon values between train & test data for column X6. 
# So we can directly apply using column
encX6 = LabelEncoder()
trn['X6'] = encX6.fit_transform(trn['X6'])
tst['X6'] = encX6.transform(tst['X6'])

In [44]:
# Now we will identify the uncommon values for X8 column
print(set(trn['X8'].unique()) - (set(tst['X8'].unique())))
print(set(tst['X8'].unique()) - (set(trn['X8'].unique())))

set()
set()


In [45]:
# From the above we can see there are no uncommon values between train & test data for column X8. 
# So we can directly apply using column
encX8 = LabelEncoder()
trn['X8'] = encX8.fit_transform(trn['X8'])
tst['X8'] = encX8.transform(tst['X8'])

In [46]:
trn.head()

,y,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,130.81,37,23,20,0,3,27,9,14,0,...,0,0,1,0,0,0,0,0,0,0
1,88.53,37,21,22,4,3,31,11,14,0,...,1,0,0,0,0,0,0,0,0,0
2,76.26,24,24,38,2,3,30,9,23,0,...,0,0,0,0,0,0,1,0,0,0
3,80.62,24,21,38,5,3,30,11,4,0,...,0,0,0,0,0,0,0,0,0,0
4,78.02,24,23,38,5,3,14,3,13,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
# Identify features with 0 variance , Since we could not find variance of categorical features, we do this after label encoding
temp = []
for i in trn.columns:
    if trn[i].var()==0:
        temp.append(i)
        
print('No. of features in train data has zero variance:',len(temp))
print('List here:',temp)

No. of features in train data has zero variance: 12
List here: ['X11', 'X93', 'X107', 'X233', 'X235', 'X268', 'X289', 'X290', 'X293', 'X297', 'X330', 'X347']


In [48]:
# Dropping cols with Zero variance

trn.drop(['X11', 'X93', 'X107', 'X233', 'X235', 'X268', 'X289', 'X290', 'X293', 'X297', 'X330', 'X347'], axis=1)

,y,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,130.81,37,23,20,0,3,27,9,14,0,...,0,0,1,0,0,0,0,0,0,0
1,88.53,37,21,22,4,3,31,11,14,0,...,1,0,0,0,0,0,0,0,0,0
2,76.26,24,24,38,2,3,30,9,23,0,...,0,0,0,0,0,0,1,0,0,0
3,80.62,24,21,38,5,3,30,11,4,0,...,0,0,0,0,0,0,0,0,0,0
4,78.02,24,23,38,5,3,14,3,13,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,107.39,10,20,19,2,3,1,3,16,0,...,1,0,0,0,0,0,0,0,0,0
4205,108.77,36,16,44,3,3,1,7,7,0,...,0,1,0,0,0,0,0,0,0,0
4206,109.22,10,23,42,0,3,1,6,4,0,...,0,0,1,0,0,0,0,0,0,0
4207,87.48,11,19,29,5,3,1,11,20,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
# Since features with 0 variance are removed from train data, the same should be removed from test data

tst.drop(['X11', 'X93', 'X107', 'X233', 'X235', 'X268', 'X289', 'X290', 'X293', 'X297', 'X330', 'X347'], axis=1)

,X0,X1,X2,X3,X4,X5,X6,X8,X10,X12,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,24,23,38,5,3,26,0,22,0,0,...,0,0,0,1,0,0,0,0,0,0
1,46,3,9,0,3,9,6,24,0,0,...,0,0,1,0,0,0,0,0,0,0
2,24,23,19,5,3,0,9,9,0,0,...,0,0,0,1,0,0,0,0,0,0
3,24,13,38,5,3,32,11,13,0,0,...,0,0,0,1,0,0,0,0,0,0
4,49,20,19,2,3,31,8,12,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4204,9,9,19,5,3,1,9,4,0,0,...,0,0,0,0,0,0,0,0,0,0
4205,46,1,9,3,3,1,9,24,0,0,...,0,1,0,0,0,0,0,0,0,0
4206,51,23,19,5,3,1,3,22,0,0,...,0,0,0,0,0,0,0,0,0,0
4207,10,23,19,0,3,1,2,16,0,0,...,0,0,1,0,0,0,0,0,0,0


In [50]:
# Checking whether all features in train are numerical datatype to go before PCA
dtype_df = trn.dtypes.reset_index()
dtype_df.columns = ["feature name","dtypes"]
dtype_df.groupby("dtypes").agg("count").reset_index()

,dtypes,feature name
0,int32,8
1,int64,368
2,float64,1


In [51]:
# Checking whether all features in test are numerical datatype to go before PCA
dtype_df = tst.dtypes.reset_index()
dtype_df.columns = ["feature name","dtypes"]
dtype_df.groupby("dtypes").agg("count").reset_index()

,dtypes,feature name
0,int32,8
1,int64,368


In [52]:
# Dropping ID from both train & test as it will not be used by model

trnPca=trn.drop(['y'],axis=1)
tstPca=tst

In [53]:
# Perform dimensionality reduction using PCA
from sklearn.decomposition import PCA
n_comp = 12
pca = PCA(n_components=n_comp, random_state=420)
trnPca= pca.fit_transform(trnPca)
tstPca = pca.transform(tstPca)

In [54]:
trnPca.shape

(4209, 12)

In [55]:
tstPca.shape

(4209, 12)

In [56]:
# Now the total 377 X columns are reduced into 12 columns after applying PCA

In [57]:
# ML Modeling with XGboost
import xgboost as xgb
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

# Defining train & test for model input
train_X = trnPca
train_y = trn['y']

# Splitting
x_train, x_valid, y_train, y_valid = train_test_split(train_X, train_y, test_size=0.2, random_state=420)

# Defining feature set
d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)
d_test = xgb.DMatrix(tstPca)
xgb_params = {
 'n_trees': 500, 
 'eta': 0.0050,
 'max_depth': 3,
 'subsample': 0.95,
 'objective': 'reg:linear',
 'eval_metric': 'rmse',
 'base_score': np.mean(train_y), # base prediction = mean(target)
 'silent': 1
}

# Creating a function for the predicting score
def xgb_r2_score(preds, dtrain):
 labels = dtrain.get_label()
 return 'r2', r2_score(labels, preds)
watchlist = [(d_train, 'train'), (d_valid, 'valid')]
mdl = xgb.train(xgb_params, d_train, 1050 , watchlist, early_stopping_rounds=50, feval=xgb_r2_score, maximize=True, verbose_eval=10)

[0]	train-rmse:12.76632	train-r2:0.00282	valid-rmse:12.22346	valid-r2:0.00315
[10]	train-rmse:12.58994	train-r2:0.03018	valid-rmse:12.03918	valid-r2:0.03298
[20]	train-rmse:12.42778	train-r2:0.05501	valid-rmse:11.86667	valid-r2:0.06049
[30]	train-rmse:12.27615	train-r2:0.07793	valid-rmse:11.70943	valid-r2:0.08522


C:\Users\private admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\xgboost\core.py:727: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)
C:\Users\private admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\xgboost\training.py:38: UserWarning: `feval` is deprecated, use `custom_metric` instead.  They have different behavior when custom objective is also used.See https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html for details on the `custom_metric`.
  warnings.warn(
C:\Users\private admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\xgboost\core.py:160: UserWarning: [17:39:19] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src

[40]	train-rmse:12.13594	train-r2:0.09887	valid-rmse:11.56285	valid-r2:0.10798
[50]	train-rmse:12.00744	train-r2:0.11785	valid-rmse:11.42841	valid-r2:0.12861
[60]	train-rmse:11.88739	train-r2:0.13540	valid-rmse:11.30409	valid-r2:0.14746
[70]	train-rmse:11.77693	train-r2:0.15139	valid-rmse:11.18683	valid-r2:0.16506
[80]	train-rmse:11.67307	train-r2:0.16630	valid-rmse:11.07694	valid-r2:0.18138
[90]	train-rmse:11.57470	train-r2:0.18029	valid-rmse:10.97360	valid-r2:0.19658
[100]	train-rmse:11.48251	train-r2:0.19329	valid-rmse:10.87597	valid-r2:0.21082
[110]	train-rmse:11.39772	train-r2:0.20516	valid-rmse:10.78788	valid-r2:0.22355
[120]	train-rmse:11.31858	train-r2:0.21616	valid-rmse:10.70671	valid-r2:0.23519
[130]	train-rmse:11.24455	train-r2:0.22638	valid-rmse:10.63086	valid-r2:0.24599
[140]	train-rmse:11.17504	train-r2:0.23592	valid-rmse:10.56167	valid-r2:0.25577
[150]	train-rmse:11.11121	train-r2:0.24462	valid-rmse:10.49609	valid-r2:0.26498
[160]	train-rmse:11.05126	train-r2:0.25275	val

[300]	train-rmse:10.42880	train-r2:0.33456	valid-rmse:9.83753	valid-r2:0.35432
[310]	train-rmse:10.39135	train-r2:0.33933	valid-rmse:9.80544	valid-r2:0.35853
[320]	train-rmse:10.36112	train-r2:0.34317	valid-rmse:9.77928	valid-r2:0.36195
[330]	train-rmse:10.32914	train-r2:0.34721	valid-rmse:9.75321	valid-r2:0.36534
[340]	train-rmse:10.29690	train-r2:0.35128	valid-rmse:9.72447	valid-r2:0.36908
[350]	train-rmse:10.26592	train-r2:0.35518	valid-rmse:9.69836	valid-r2:0.37246
[360]	train-rmse:10.23622	train-r2:0.35891	valid-rmse:9.67313	valid-r2:0.37572
[370]	train-rmse:10.20392	train-r2:0.36295	valid-rmse:9.64734	valid-r2:0.37905
[380]	train-rmse:10.17518	train-r2:0.36653	valid-rmse:9.62368	valid-r2:0.38209
[390]	train-rmse:10.14940	train-r2:0.36974	valid-rmse:9.60276	valid-r2:0.38477
[400]	train-rmse:10.12067	train-r2:0.37330	valid-rmse:9.57803	valid-r2:0.38794
[410]	train-rmse:10.09396	train-r2:0.37660	valid-rmse:9.55632	valid-r2:0.39071
[420]	train-rmse:10.06958	train-r2:0.37961	valid-rms

In [58]:
# Predicting on test set
p_test = mdl.predict(d_test)
p_test

array([ 77.044106,  96.41954 ,  83.96951 , ..., 102.60102 , 107.38367 ,
        96.2188  ], dtype=float32)

In [59]:
Predicted_Data = pd.DataFrame()
Predicted_Data['y'] = p_test
Predicted_Data.head()

,y
0,77.044106
1,96.419540
2,83.969513
3,77.468704
4,110.402405


In [60]:
# With the above model, we have the below metrics:
#train-rmse:9.15415	
#train-r2:0.48728	
#valid-rmse:8.99823	
#valid-r2:0.45980